In [4]:
import os
import sys
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
import toml
import json
import re

# Load config
project_root = Path('.').resolve().parent.parent
config = toml.load(project_root / 'config.toml')
dial_config = config.get('DIAL_API', {})

dial_api_endpoint = dial_config.get('DIAL_API_ENDPOINT', 'https://ai-proxy.lab.epam.com/openai/deployments/{model_id}/chat/completions')
dial_models_endpoint = 'https://ai-proxy.lab.epam.com/openai/models'
api_key = os.getenv('DIAL_API_KEY')

print('✅ Configuration loaded')
print(f'   API Key: {bool(api_key)}')

✅ Configuration loaded
   API Key: True


In [5]:
# Retrieve ALL models
print('Fetching models...')
headers = {'api-key': api_key}
response = requests.get(dial_models_endpoint, headers=headers, timeout=10)

if response.status_code == 200:
    all_models = response.json().get('data', [])
    print(f'✅ Retrieved {len(all_models)} models\n')
    
    # Analyze types
    model_types = {}
    for model in all_models:
        mtype = model.get('type', 'unknown')
        model_types[mtype] = model_types.get(mtype, 0) + 1
    
    print('Model types:')
    for mtype, count in sorted(model_types.items(), key=lambda x: -x[1]):
        print(f'  {mtype:<20} {count:3d}')
else:
    print(f'Error: {response.status_code}')

Fetching models...
✅ Retrieved 98 models

Model types:
  unknown               98


In [6]:
# Filter for text-generating models
def is_text_gen(model_id: str) -> bool:
    exclude = ['embedding', 'embed', 'vision-', 'image', 'dall-e', 'whisper', 'tts', 'audio']
    include = ['gpt', 'claude', 'gemini', 'llama', 'mistral', 'deepseek', 'phi', 'qwen']
    
    model_lower = model_id.lower()
    if any(p in model_lower for p in exclude):
        return False
    return any(p in model_lower for p in include)

text_gen = [m for m in all_models if is_text_gen(m.get('id', ''))]
text_gen = sorted(text_gen, key=lambda x: x.get('id', ''))

print(f'\nFiltered: {len(text_gen)} text-generating models\n')
print('Models:')
for i, m in enumerate(text_gen, 1):
    print(f'  {i:2d}. {m.get("id")}')


Filtered: 73 text-generating models

Models:
   1. anthropic.claude-3-7-sonnet-20250219-v1:0
   2. anthropic.claude-3-7-sonnet-20250219-v1:0-with-thinking
   3. anthropic.claude-haiku-4-5-20251001-v1:0
   4. anthropic.claude-haiku-4-5-20251001-v1:0-with-thinking
   5. anthropic.claude-opus-4-1-20250805-v1:0
   6. anthropic.claude-opus-4-1-20250805-v1:0-with-thinking
   7. anthropic.claude-opus-4-20250514-v1:0
   8. anthropic.claude-opus-4-20250514-v1:0-with-thinking
   9. anthropic.claude-opus-4-5-20251101-v1:0
  10. anthropic.claude-opus-4-5-20251101-v1:0-with-thinking
  11. anthropic.claude-sonnet-4-20250514-v1:0
  12. anthropic.claude-sonnet-4-20250514-v1:0-with-thinking
  13. anthropic.claude-sonnet-4-5-20250929-v1:0
  14. anthropic.claude-sonnet-4-5-20250929-v1:0-with-thinking
  15. anthropic.claude-v3-5-haiku
  16. anthropic.claude-v3-5-sonnet
  17. anthropic.claude-v3-5-sonnet-v1
  18. anthropic.claude-v3-5-sonnet-v2
  19. anthropic.claude-v3-haiku
  20. anthropic.claude-v3-opu

In [ ]:
# Test models
def test_model(model_id: str) -> dict:
    url = dial_api_endpoint.format(model_id=model_id)
    headers = {'Api-Key': api_key, 'Content-Type': 'application/json'}
    data = {'messages': [{'role': 'user', 'content': 'test'}], 'max_tokens': 5}
    
    try:
        start = datetime.now()
        response = requests.post(url, headers=headers, json=data, timeout=15)
        elapsed = (datetime.now() - start).total_seconds()
        
        if response.status_code == 200:
            return {'model': model_id, 'status': '✅ Available', 'time': round(elapsed, 2), 'code': 200}
        elif response.status_code == 403:
            return {'model': model_id, 'status': '❌ Blocked', 'time': round(elapsed, 2), 'code': 403}
        elif response.status_code == 429:
            return {'model': model_id, 'status': '⏳ Quota', 'time': round(elapsed, 2), 'code': 429}
        else:
            return {'model': model_id, 'status': f'❌ Error {response.status_code}', 'time': round(elapsed, 2), 'code': response.status_code}
    except Exception as e:
        return {'model': model_id, 'status': '❌ Error', 'time': None, 'code': None, 'error': str(e)[:50]}

print(f'Testing {len(text_gen)} models...\n')
results = []

for i, model in enumerate(text_gen, 1):
    model_id = model.get('id')
    print(f'[{i:2d}/{len(text_gen)}] {model_id}...', end=' ', flush=True)
    result = test_model(model_id)
    results.append(result)
    print(result['status'])

print(f'\n✅ Done!')

Testing 73 models...

[ 1/73] anthropic.claude-3-7-sonnet-20250219-v1:0... ❌ Blocked
[ 2/73] anthropic.claude-3-7-sonnet-20250219-v1:0-with-thinking... ❌ Blocked
[ 3/73] anthropic.claude-haiku-4-5-20251001-v1:0... ✅ Available
[ 4/73] anthropic.claude-haiku-4-5-20251001-v1:0-with-thinking... ❌ Blocked
[ 5/73] anthropic.claude-opus-4-1-20250805-v1:0... ❌ Blocked
[ 6/73] anthropic.claude-opus-4-1-20250805-v1:0-with-thinking... ❌ Blocked
[ 7/73] anthropic.claude-opus-4-20250514-v1:0... ❌ Blocked
[ 8/73] anthropic.claude-opus-4-20250514-v1:0-with-thinking... ❌ Blocked
[ 9/73] anthropic.claude-opus-4-5-20251101-v1:0... ❌ Blocked
[10/73] anthropic.claude-opus-4-5-20251101-v1:0-with-thinking... ❌ Blocked
[11/73] anthropic.claude-sonnet-4-20250514-v1:0... ❌ Blocked
[12/73] anthropic.claude-sonnet-4-20250514-v1:0-with-thinking... ❌ Blocked
[13/73] anthropic.claude-sonnet-4-5-20250929-v1:0... ❌ Blocked
[14/73] anthropic.claude-sonnet-4-5-20250929-v1:0-with-thinking... ❌ Blocked
[15/73] anthropic.

In [8]:
# Summary
df = pd.DataFrame(results)
status_counts = df['status'].value_counts()

print('\n' + '='*60)
print('SUMMARY')
print('='*60)
for status, count in status_counts.items():
    print(f'{status:20s} {count:3d}')

print(f'\nTotal: {len(df)} models tested')


SUMMARY
❌ Blocked             50
✅ Available           19
❌ Error 400            3
❌ Error                1

Total: 73 models tested


In [9]:
# Show blocked model responses
print('\n' + '='*60)
print('BLOCKED MODELS - RESPONSE DETAILS')
print('='*60)

blocked_models = [r['model'] for r in results if '❌ Blocked' in r['status']]
print(f'\nFound {len(blocked_models)} blocked models\n')

for model_id in blocked_models[:5]:  # Show first 5
    url = dial_api_endpoint.format(model_id=model_id)
    headers = {'Api-Key': api_key, 'Content-Type': 'application/json'}
    data = {'messages': [{'role': 'user', 'content': 'test'}], 'max_tokens': 5}
    
    try:
        response = requests.post(url, headers=headers, json=data, timeout=15)
        print(f'\n{model_id}:')
        print(f'  Status Code: {response.status_code}')
        print(f'  Headers: {dict(response.headers)}')
        try:
            print(f'  Body: {response.json()}')
        except:
            print(f'  Body (text): {response.text[:200]}')
    except Exception as e:
        print(f'\n{model_id}: Error - {e}')



BLOCKED MODELS - RESPONSE DETAILS

Found 50 blocked models


anthropic.claude-3-7-sonnet-20250219-v1:0:
  Status Code: 403
  Headers: {'Date': 'Mon, 15 Dec 2025 10:08:34 GMT', 'Content-Length': '83', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'content-encoding': 'gzip', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}
  Body: {'error': {'message': 'Access denied', 'display_message': 'Access denied', 'code': '403'}}

anthropic.claude-3-7-sonnet-20250219-v1:0-with-thinking:
  Status Code: 403
  Headers: {'Date': 'Mon, 15 Dec 2025 10:08:35 GMT', 'Content-Length': '83', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'content-encoding': 'gzip', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}
  Body: {'error': {'message': 'Access denied', 'display_message': 'Access denied', 'code': '403'}}

anthropic.claude-haiku-4-5-20251001-v1:0-with-thinking:
  Status Code: 403
  Headers: {'Date': 'Mon, 15 Dec 2025 10:08:35 GMT'